In [2]:
def interactive_earthquake_plot():
    @interact(i_t=widgets.IntSlider(min=0, max=100, value=25, step=25, description='$t/T$: [%]'),
              i_f=widgets.IntSlider(min=5, max=100, value=50, step=5, description='$f$ [dHz]:'),
              i_d=widgets.IntSlider(min=0, max=100, value=0, step=10, description='$d/h$: [%]')
             )

    def plot_now(i_t, i_f, i_d):
        # example
        CENT = 100  # for perCENT
        c_n = 150.0   
        f_n = i_f/10.0
        omega_n = 2*np.pi*f_n 
        T_n = 1/f_n
        kappa_n = omega_n/c_n
        c2_n = 3000.0
        kappa2_n = omega_n/c2_n
        L_n = 2*np.pi/kappa2_n
        h_n = 10.0
        d_n = (i_d/CENT)*h_n 
        u0_n = 0.1
        
        # find lower eigenfrequencies
        ch_eq = lambda xx: (d_n/h_n)*xx - 1/np.tan(xx)
        x_guess = (np.pi/2)*(1-np.tanh(d_n/h_n))   # 0...pi/2
        x_num = fsolve(ch_eq, x_guess)
        omega_0 = x_num[0]*c_n/h_n
        print("Eigenfrequency f0(d)={} dHz".format(10*omega_0/(2*np.pi)))
        x_guess = np.pi + (np.pi/2)*(1-np.tanh(2*d_n/h_n))   # pi...3pi/2
        x_num = fsolve(ch_eq, x_guess)
        omega_1 = x_num[0]*c_n/h_n
        print("Eigenfrequency f1(d)={} dHz".format(10*omega_1/(2*np.pi)))
        x_guess = 2*np.pi + (np.pi/2)*(1-np.tanh(3*d_n/h_n))   # 2pi...5pi/2
        x_num = fsolve(ch_eq, x_guess)
        omega_2 = x_num[0]*c_n/h_n
        print("Eigenfrequency f2(d)={} dHz".format(10*omega_2/(2*np.pi)))
        
        
        example = [ (omega, omega_n), (kappa, kappa_n), (kappa2, kappa2_n), (h, h_n), (d, d_n), (u0, u0_n) ]
        w_example = sp.lambdify((x, z, t), wsol.subs(example), modules='numpy' )
        
        N = 50
        z_n = np.linspace(0, h_n, N)
        t_n = (i_t/CENT)*T_n
        scale_factor=(L_n/u0_n)*5e-3
        
        Columns = 10
        for C in np.arange(Columns+1):
            X = (C/Columns)*L_n
            w_n = w_example(X, z_n, t_n)*scale_factor     
            plt.plot([X, X], [0, -h_n], 'k--', X+w_n, -z_n, 'k')
        
        plt.xlabel('$x$')
        plt.ylabel('$z$')
        plt.show()
        